In [64]:
#!pip install checklist

In [65]:
#!pip install simpletransformers

In [66]:
import numpy as np
import checklist
import pandas as pd
import spacy
from checklist.editor import Editor
from checklist.perturb import Perturb
from simpletransformers.classification import ClassificationModel,ClassificationArgs
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix,average_precision_score
import warnings
warnings.filterwarnings("ignore")

In [67]:
data = pd.read_csv('/content/data/olid-subset-diagnostic-tests.csv')
train_data = pd.read_csv('/content/data/olid-train.csv')

data = data[['text','labels']]
train_data = train_data[['text','labels']]
data.head(3)

,text,labels
0,@USER @USER Who the hell does he think he is?,1
1,#BREAKING. #Greece: Molotov cocktails fly afte...,1
2,"#OrrinHatch I can’t believe this sexist , clue...",1


In [68]:
nlp = spacy.load('en_core_web_sm')

In [69]:
perturbed_data = list(nlp.pipe(data['text']))

In [70]:
perturbed_data[:3]

[@USER @USER Who the hell does he think he is?,
 #BREAKING. #Greece: Molotov cocktails fly after protest honouring killed antifa arti... URL via @USER URL,
 #OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!]

In [71]:
np.random.seed(42)
ret_data_typo2 = Perturb.perturb(data['text'], Perturb.add_typos, typos = 2)
ret_data_typo4 = Perturb.perturb(data['text'], Perturb.add_typos, typos = 4)
ret_data_typo6 = Perturb.perturb(data['text'], Perturb.add_typos, typos = 6)
ret_data_typo8 = Perturb.perturb(data['text'], Perturb.add_typos, typos = 8)
ret_data_typo10 = Perturb.perturb(data['text'], Perturb.add_typos, typos = 10)

ret_data_typo2.data[:3][1]

['#BREAKING. #Greece: Molotov cocktails fly after protest honouring killed antifa arti... URL via @USER URL',
 '#BREAKING. #Greece: Molotov cocktails fly after protest honouring kille dantifa arti... URL via @USER URL']

In [72]:
def make_data_list(data):
  typo_data = []
  for i in range(len(data)):
    typo_data.append(data[i][1]) # tweet with typos added
  
  return typo_data

In [73]:
typo_data2 = make_data_list(ret_data_typo2.data)
typo_data4 = make_data_list(ret_data_typo4.data)
typo_data6 = make_data_list(ret_data_typo6.data)
typo_data8 = make_data_list(ret_data_typo8.data)
typo_data10 = make_data_list(ret_data_typo10.data)

In [74]:
print(typo_data2[0])
print(typo_data10[0])

@USER @USER Who the hell doe she thinkh e is?
U@SER U@SE RhWo th eehl dloes he think h eis?


In [75]:
data_typo2 = pd.DataFrame({'text':typo_data2,'label':data['labels']})
data_typo4 = pd.DataFrame({'text':typo_data4,'label':data['labels']})
data_typo6 = pd.DataFrame({'text':typo_data6,'label':data['labels']})
data_typo8 = pd.DataFrame({'text':typo_data8,'label':data['labels']})
data_typo10 = pd.DataFrame({'text':typo_data10,'label':data['labels']})

data_typo2.head()

,text,label
0,@USER @USER Who the hell doe she thinkh e is?,1
1,#BREAKING. #Greece: Molotov cocktails fly afte...,1
2,"#OrrinHatch I can’t eblieve this sexist , clue...",1
3,@USER @USER I'll use that one the next time im...,1
4,0-1 lost my acca o nth efirst fucking fight cba,1


In [76]:
model_args = ClassificationArgs()
model_args.num_train_epochs = 1
model_args.overwrite_output_dir= True
model_args.output_dir = '/content/drive/MyDrive/NLP/error_analyse_model'


model = ClassificationModel("bert", 'bert-base-cased', args=model_args)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [77]:
model.train_model(train_data)

  0%|          | 0/13240 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/1655 [00:00<?, ?it/s]

(1655, 0.4998074482574924)

In [78]:
predictions2, raw_outputs2 = model.predict(list(data_typo2['text']))
predictions4, raw_outputs4 = model.predict(list(data_typo4['text']))
predictions6, raw_outputs6 = model.predict(list(data_typo6['text']))
predictions8, raw_outputs8 = model.predict(list(data_typo8['text']))
predictions10, raw_outputs10 = model.predict(list(data_typo10['text']))

pred, out = model.predict(list(data['text']))

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [79]:
predicted_typo_labels2 = pd.DataFrame({'text':data_typo2.text, 'labels': predictions2})
predicted_typo_labels2.head()

,text,labels
0,@USER @USER Who the hell doe she thinkh e is?,1
1,#BREAKING. #Greece: Molotov cocktails fly afte...,0
2,"#OrrinHatch I can’t eblieve this sexist , clue...",1
3,@USER @USER I'll use that one the next time im...,0
4,0-1 lost my acca o nth efirst fucking fight cba,1


In [80]:
predicted_typo_labels10 = pd.DataFrame({'text':data_typo10.text, 'labels': predictions10})
predicted_typo_labels10.head()

,text,labels
0,U@SER U@SE RhWo th eehl dloes he think h eis?,0
1,#BREAKING. #Greece :oMlotov cocktails fly afte...,0
2,"#OrrinHatch I can’t believe this sexist , clue...",1
3,@USER @USERI 'll uset hat one the next tim eim...,0
4,0-1l ost m ycaac o nhet first fucikgn fight cba,0


In [81]:
result2, model_outputs2, wrong_predictions2 = model.eval_model(data_typo2, report_typo2 = classification_report)
result4, model_outputs4, wrong_predictions4 = model.eval_model(data_typo4, report_typo4 = classification_report)
result6, model_outputs6, wrong_predictions6 = model.eval_model(data_typo6, report_typo6 = classification_report)
result8, model_outputs8, wrong_predictions8 = model.eval_model(data_typo8, report_typo8 = classification_report)
result10, model_outputs10, wrong_predictions10 = model.eval_model(data_typo10, report_typo10 = classification_report)

result_d, model_outputs_d, wrong_predictions_d = model.eval_model(data, report_normaldata = classification_report)

  0%|          | 0/100 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

In [82]:
print("Original data prediction")
print(result_d['report_normaldata'])
print("\n")
print("With 2 typos")
print(result2['report_typo2'])
print("\n")
print("With 4 typos")
print(result4['report_typo4'])
print("\n")
print("With 6 typos")
print(result6['report_typo6'])
print("\n")
print("With 8 typos")
print(result8['report_typo8'])
print("\n")
print("With 10 typos")
print(result10['report_typo10'])


Original data prediction
              precision    recall  f1-score   support

         0.0       0.71      0.98      0.82        50
         1.0       0.97      0.60      0.74        50

    accuracy                           0.79       100
   macro avg       0.84      0.79      0.78       100
weighted avg       0.84      0.79      0.78       100



With 2 typos
              precision    recall  f1-score   support

         0.0       0.71      0.98      0.82        50
         1.0       0.97      0.60      0.74        50

    accuracy                           0.79       100
   macro avg       0.84      0.79      0.78       100
weighted avg       0.84      0.79      0.78       100



With 4 typos
              precision    recall  f1-score   support

         0.0       0.66      1.00      0.79        50
         1.0       1.00      0.48      0.65        50

    accuracy                           0.74       100
   macro avg       0.83      0.74      0.72       100
weighted avg       

In [83]:
print(model_outputs10[:5])

[[ 0.86132812 -1.28515625]
 [ 0.60742188 -0.91601562]
 [-1.20214844  1.23632812]
 [ 0.01368713 -0.56054688]
 [ 0.83447266 -1.22460938]]


In [84]:
print("Confusion Matrix (2 typos): ")
print(confusion_matrix(data['labels'], predictions2))
print("\n")
print("Confusion Matrix (4 typos): ")
print(confusion_matrix(data['labels'], predictions4))
print("\n")
print("Confusion Matrix (6 typos): ")
print(confusion_matrix(data['labels'], predictions6))
print("\n")
print("Confusion Matrix (8 typos): ")
print(confusion_matrix(data['labels'], predictions8))
print("\n")
print("Confusion Matrix (10 typos): ")
print(confusion_matrix(data['labels'], predictions10))
print("\n")

Confusion Matrix (2 typos): 
[[49  1]
 [20 30]]


Confusion Matrix (4 typos): 
[[50  0]
 [26 24]]


Confusion Matrix (6 typos): 
[[48  2]
 [28 22]]


Confusion Matrix (8 typos): 
[[50  0]
 [25 25]]


Confusion Matrix (10 typos): 
[[48  2]
 [31 19]]




### Provide 3 examples when the model failed to assign the correct label after perturbation.

In [85]:
print(data['labels'][:10])

0    1
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
9    1
Name: labels, dtype: int64


In [86]:
print(predicted_typo_labels10['labels'][:10])

0    0
1    0
2    1
3    0
4    0
5    1
6    0
7    1
8    0
9    1
Name: labels, dtype: int64


In [87]:
failed_texts=[]
failed_labels=[]
for i in range(len(data)):
  if predicted_typo_labels10['labels'][i] != data['labels'][i]:
    failed_texts.append(predicted_typo_labels10['text'][i])
    failed_labels.append(predicted_typo_labels10['labels'][i])

In [88]:
failed_examples = pd.DataFrame({'text':failed_texts, 'labels': failed_labels})
failed_examples

,text,labels
0,U@SER U@SE RhWo th eehl dloes he think h eis?,0
1,#BREAKING. #Greece :oMlotov cocktails fly afte...,0
2,@USER @USERI 'll uset hat one the next tim eim...,0
3,0-1l ost m ycaac o nhet first fucikgn fight cba,0
4,#hCristian #America – If we go by #Trump’s exa...,0
5,#Democrats #Liberals you are being t#hreaetned...,0
6,#LiberalHpyocrsiy #criminal #democrat #evidenc...,0
7,(crt1ikal voice) emsf m ayss chekes,0
8,t*ge sall teh ibthce*s,0
9,@U SREof hucko ff 😂😂,0


# 6. Negation *(4.5 points)* 

In [89]:
perturbed_data = list(nlp.pipe(data['text']))

In [90]:
ret = Perturb.perturb(perturbed_data, Perturb.add_negation) # negated data
ret.data[:5]

[['@USER @USER Who the hell does he think he is?',
  "@USER @USER Who the hell doesn't he think he is?"],
 ['#BREAKING. #Greece: Molotov cocktails fly after protest honouring killed antifa arti... URL via @USER URL',
  "#BREAKING. #Greece: Molotov cocktails don't fly after protest honouring killed antifa arti... URL via @USER URL"],
 ['#OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!',
  '#OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is not spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!'],
 ["@USER @USER I'll use that one the next time im in a gun control debate or in a debate about free speech or taxes.

In [91]:
negated_text = []
for i in range(len(ret.data)):
  negated_text.append(ret.data[i][1])

negated_text[:5]

["@USER @USER Who the hell doesn't he think he is?",
 "#BREAKING. #Greece: Molotov cocktails don't fly after protest honouring killed antifa arti... URL via @USER URL",
 '#OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is not spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!',
 "@USER @USER I'll use that one the next time im in a gun control debate or in a debate about free speech or taxes. Yes you can't choose to be irresponsible or choose not to be. I argue responsible. Whats wrong with that? Don't justify murder by saying it was never alive or its my right.",
 "0-1 didn't lose my acca on the first fucking fight cba"]

In [92]:
neg_predictions, neg_raw_outputs = model.predict(negated_text)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [93]:
predicted_neg_labels = pd.DataFrame({'text':negated_text, 'labels': data['labels']})
predicted_neg_labels.head()

,text,labels
0,@USER @USER Who the hell doesn't he think he is?,1
1,#BREAKING. #Greece: Molotov cocktails don't fl...,1
2,"#OrrinHatch I can’t believe this sexist , clue...",1
3,@USER @USER I'll use that one the next time im...,1
4,0-1 didn't lose my acca on the first fucking f...,1


In [94]:
result, model_outputs, wrong_predictions = model.eval_model(predicted_neg_labels, neg_report = classification_report)

  0%|          | 0/100 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

In [95]:
print(result['neg_report'])

              precision    recall  f1-score   support

         0.0       0.71      0.98      0.82        50
         1.0       0.97      0.60      0.74        50

    accuracy                           0.79       100
   macro avg       0.84      0.79      0.78       100
weighted avg       0.84      0.79      0.78       100



In [96]:
print("Confusion Matrix : ")
print(confusion_matrix(data['labels'], neg_predictions))

Confusion Matrix : 
[[49  1]
 [20 30]]


### Check the first 10 negated messages. For which of these negated messages should the label be flipped, in your opinion?

In [97]:
predicted_neg_labels['text'][:10]

0     @USER @USER Who the hell doesn't he think he is?
1    #BREAKING. #Greece: Molotov cocktails don't fl...
2    #OrrinHatch I can’t believe this sexist , clue...
3    @USER @USER I'll use that one the next time im...
4    0-1 didn't lose my acca on the first fucking f...
5    #Bakersfield is not why we need gun control! S...
6    #Christian #America – If we go by #Trump’s exa...
7    @USER @USER @USER She is not the most disingen...
8    #Democrats #Liberals you are being #threatened...
9    699. Just didn't want to tell you you should H...
Name: text, dtype: object

In [98]:
predicted_neg_labels_df = pd.DataFrame({'text':negated_text, 'labels': neg_predictions})
predicted_neg_labels_df.head()

,text,labels
0,@USER @USER Who the hell doesn't he think he is?,1
1,#BREAKING. #Greece: Molotov cocktails don't fl...,0
2,"#OrrinHatch I can’t believe this sexist , clue...",1
3,@USER @USER I'll use that one the next time im...,0
4,0-1 didn't lose my acca on the first fucking f...,1


In [99]:
failed_neg_texts=[]
failed_neg_labels=[]
for i in range(len(data)):
  if predicted_neg_labels_df['labels'][i] != data['labels'][i]:
    failed_neg_texts.append(predicted_neg_labels['text'][i])
    failed_neg_labels.append(predicted_neg_labels['labels'][i])

In [100]:
failed_neg_examples = pd.DataFrame({'text':failed_neg_texts, 'labels': failed_neg_labels})
failed_neg_examples

,text,labels
0,#BREAKING. #Greece: Molotov cocktails don't fl...,1
1,@USER @USER I'll use that one the next time im...,1
2,#Christian #America – If we go by #Trump’s exa...,1
3,#Democrats #Liberals you are being #threatened...,1
4,#LiberalHypocrisy #criminal #democrat #evidenc...,1
5,#Antifa: don't take note of how to protest wit...,1
6,#Nigeria #Naija #9ja 'You are not the most inc...,1
7,#ANTIFA are not bedfellows with The Democratic...,1
8,#LifeIsButepu when you have sex anytime you wa...,1
9,@USER Antifa doesn't have TS level influence. ...,1


In [101]:
success_neg_texts=[]
success_neg_labels=[]
for i in range(len(data)):
  if predicted_neg_labels_df['labels'][i] == data['labels'][i]:
    success_neg_texts.append(predicted_neg_labels['text'][i])
    success_neg_labels.append(predicted_neg_labels['labels'][i])

In [102]:
successful_neg_examples = pd.DataFrame({'text':success_neg_texts, 'labels': success_neg_labels})
successful_neg_examples

,text,labels
0,@USER @USER Who the hell doesn't he think he is?,1
1,"#OrrinHatch I can’t believe this sexist , clue...",1
2,0-1 didn't lose my acca on the first fucking f...,1
3,#Bakersfield is not why we need gun control! S...,1
4,@USER @USER @USER She is not the most disingen...,1
...,...,...
74,@USER He is obviously getting not suspended. H...,0
75,#Canada - EXCLUSIVE: #Trudeau #Liberals don't ...,0
76,@USER @USER ...than why didn't you show us how...,0
77,@USER @USER @USER You don't have yet to answer...,0


## 7. Creating negated examples